- V1 : subsector_id  
- V2 : merchant_category_id
- V3 : city_id
- V4 : merchant_category_id + TRICK
- V6 : v2 + TRICK
- V7 : TRICK 제거 + -1의 갯수 
- V8 : -1인 정보 모두 제거?

In [26]:
import gc
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error, log_loss
from tqdm import tqdm

In [2]:
#settings
warnings.filterwarnings('ignore')
np.random.seed(2018)
version = 7
#logger
def get_logger():
    FORMAT = '[%(levelname)s]%(asctime)s:%(name)s:%(message)s'
    logging.basicConfig(format=FORMAT)
    logger = logging.getLogger('main')
    logger.setLevel(logging.DEBUG)
    return logger

In [3]:
# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
logger = get_logger()
#load data
logger.info('Start read data')
train_df = reduce_mem_usage(pd.read_csv('input/train.csv'))
test_df = reduce_mem_usage(pd.read_csv('input/test.csv'))

[INFO]2019-02-06 10:19:18,867:main:Start read data


Memory usage after optimization is: 4.04 MB
Decreased by 56.2%
Memory usage after optimization is: 2.24 MB
Decreased by 52.5%


In [5]:
historical_trans_df = reduce_mem_usage(pd.read_csv('input/historical_transactions.csv'))
new_merchant_trans_df = reduce_mem_usage(pd.read_csv('input/new_merchant_transactions.csv'))

#process NAs
logger.info('Start processing NAs')
#process NAs for merchant

#process NA2 for transactions
for df in [historical_trans_df, new_merchant_trans_df]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    
#define function for aggregation
def create_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

Memory usage after optimization is: 1749.11 MB
Decreased by 43.7%


[INFO]2019-02-06 10:20:41,360:main:Start processing NAs


Memory usage after optimization is: 114.20 MB
Decreased by 45.5%


In [6]:
# -1의 갯수 
historical_trans_df['none_cnt'] = 0 
historical_trans_df.loc[(historical_trans_df['city_id']==-1) | (historical_trans_df['installments']==-1) | (historical_trans_df['merchant_category_id']==-1)
                       | (historical_trans_df['state_id']==-1) | (historical_trans_df['subsector_id']==-1), 'none_cnt'] = 1

df = historical_trans_df[['card_id','none_cnt']]
df = df.groupby('card_id')['none_cnt'].agg({'mean','var'}).reset_index()
df.columns = ['card_id','hist_none_cnt_var','hist_none_cnt_mean']
train_df = pd.merge(train_df,df,how='left',on='card_id')
test_df = pd.merge(test_df,df,how='left',on='card_id')

del df
del historical_trans_df['none_cnt']
gc.collect()

91

In [7]:
feature = 'merchant_category_id'
uniquecardidcity = historical_trans_df.groupby(['card_id'])[feature].unique().reset_index()
uniquecardidcity['histset_{}'.format(feature)] = uniquecardidcity[feature].apply(set)
newhistuniquecardidcity = new_merchant_trans_df.groupby(['card_id'])[feature].unique().reset_index() 
newhistuniquecardidcity['newhistset_{}'.format(feature)] = newhistuniquecardidcity[feature].apply(set)
uniquecardidcity = uniquecardidcity.merge(newhistuniquecardidcity[['card_id','newhistset_{}'.format(feature)]], on='card_id',how='left')
uniquecardidcity_na = uniquecardidcity[uniquecardidcity['newhistset_{}'.format(feature)].isnull()]
uniquecardidcity = uniquecardidcity.dropna(axis=0)

uniquecardidcity['union'] = uniquecardidcity.apply(lambda x: len(x['histset_{}'.format(feature)].union(x['newhistset_{}'.format(feature)])), axis=1)
uniquecardidcity['hist_new_difference_{}'.format(feature)] = uniquecardidcity.apply(lambda x: len(x['histset_{}'.format(feature)].difference(x['newhistset_{}'.format(feature)])), axis=1)
uniquecardidcity['new_hist_difference_{}'.format(feature)] = uniquecardidcity.apply(lambda x: len(x['newhistset_{}'.format(feature)].difference(x['histset_{}'.format(feature)])), axis=1)
uniquecardidcity['intersection_{}'.format(feature)] = uniquecardidcity.apply(lambda x: len(x['histset_{}'.format(feature)].intersection(x['newhistset_{}'.format(feature)])), axis=1)

uniquecardidcity['hist_new_difference_{}'.format(feature)] = uniquecardidcity['hist_new_difference_{}'.format(feature)]/uniquecardidcity['union']
uniquecardidcity['new_hist_difference_{}'.format(feature)] = uniquecardidcity['new_hist_difference_{}'.format(feature)]/uniquecardidcity['union']
uniquecardidcity['intersection_{}'.format(feature)] = uniquecardidcity['intersection_{}'.format(feature)]/uniquecardidcity['union']

uniquecardidcity = uniquecardidcity[['card_id','hist_new_difference_{}'.format(feature),'new_hist_difference_{}'.format(feature),'intersection_{}'.format(feature)]]

uniquecardidcity_na['union'] = uniquecardidcity_na['histset_{}'.format(feature)].apply(lambda x : len(x))
uniquecardidcity_na['hist_new_difference_{}'.format(feature)] = 1
uniquecardidcity_na['new_hist_difference_{}'.format(feature)] = np.nan
uniquecardidcity_na['intersection_{}'.format(feature)] = np.nan
uniquecardidcity_na = uniquecardidcity_na[['card_id','hist_new_difference_{}'.format(feature),'new_hist_difference_{}'.format(feature),'intersection_{}'.format(feature)]]

unique = pd.concat([uniquecardidcity,uniquecardidcity_na])
train_df = pd.merge(train_df,unique,how='left',on='card_id')
test_df = pd.merge(test_df,unique,how='left',on='card_id')
del unique,uniquecardidcity,uniquecardidcity_na
gc.collect()

77

In [8]:
len_hist = historical_trans_df.shape[0]
hist_new_df_all = pd.concat([historical_trans_df,new_merchant_trans_df])

In [9]:
hist_new_df_all.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [10]:
def frequency_encoding(frame, col):
    freq_encoding = frame.groupby([col]).size()/frame.shape[0] 
    freq_encoding = freq_encoding.reset_index().rename(columns={0:'{}_Frequency'.format(col)})
    return frame.merge(freq_encoding, on=col, how='left')

cat_cols = ['city_id','merchant_category_id','merchant_id','state_id','subsector_id']

freq_cat_cols = ['{}_Frequency'.format(col) for col in cat_cols]

for col in tqdm(cat_cols):
    hist_new_df_all = frequency_encoding(hist_new_df_all, col)

100%|██████████| 5/5 [02:54<00:00, 34.91s/it]


In [11]:
historical_trans_df = hist_new_df_all[:len_hist]
new_merchant_trans_df = hist_new_df_all[len_hist:]
del hist_new_df_all
gc.collect()

198

feature = 'merchant_category_id'
uniquecardidcity = historical_trans_df.groupby(['card_id'])[feature].unique().reset_index()
uniquecardidcity['histcityidset'] = uniquecardidcity[feature].apply(set)
newhistuniquecardidcity = new_merchant_trans_df.groupby(['card_id'])[feature].unique().reset_index() 
newhistuniquecardidcity['newhistcityidset'] = newhistuniquecardidcity[feature].apply(set)
uniquecardidcity = uniquecardidcity.merge(newhistuniquecardidcity[['card_id','newhistcityidset']], on='card_id',how='left') 
uniquecardidcity = uniquecardidcity.dropna() # newhist에 없는 cardid drop uniquecardidcity['union'] = uniquecardidcity.apply(lambda row: row['histcityidset'].union(row['newhistcityidset']), axis=1)
uniquecardidcity['union'] = uniquecardidcity.apply(lambda row: len(row['histcityidset'].union(row['newhistcityid_set'])), axis=1)

uniquecardidcity['intersection'] = uniquecardidcity.apply(lambda row: len(row['histcityidset'].intersection(row['newhistcityid_set'])), axis=1)
uniquecardidcity['diff_hist_new_{}'.format(feature)] = uniquecardidcity.apply(lambda row: row['histcityidset'].difference(row['newhistcityid_set']), axis=1)
uniquecardidcity['diff_new_hist_{}'.format(feature)] = uniquecardidcity.apply(lambda row: row['newhistcityid_set'].difference(row['histcityidset']), axis=1)

uniquecardidcity['intersection'] = uniquecardidcity['intersection']/uniquecardidcity['union']
uniquecardidcity['diff_hist_new_{}'.format(feature)] = uniquecardidcity['diff_hist_new_{}'.format(feature)]/uniquecardidcity['union']
uniquecardidcity['diff_new_hist_{}'.format(feature)] = uniquecardidcity['diff_new_hist_{}'.format(feature)]/uniquecardidcity['union']
del uniquecardidcity['union']

uniquecardidcity['intersection'] = uniquecardidcity.apply(lambda row: len(row['histcityidset'].intersection(row['newhistcityid_set'])), axis=1)
uniquecardidcity['diff_hist_new_{}'.format(feature)] = uniquecardidcity.apply(lambda row: row['histcityidset'].difference(row['newhistcityid_set']), axis=1)
uniquecardidcity['diff_new_hist_{}'.format(feature)] = uniquecardidcity.apply(lambda row: row['newhistcityid_set'].difference(row['histcityidset']), axis=1)

uniquecardidcity['intersection'] = uniquecardidcity['intersection']/uniquecardidcity['union']
uniquecardidcity['diff_hist_new_{}'.format(feature)] = uniquecardidcity['diff_hist_new_{}'.format(feature)]/uniquecardidcity['union']
uniquecardidcity['diff_new_hist_{}'.format(feature)] = uniquecardidcity['diff_new_hist_{}'.format(feature)]/uniquecardidcity['union']
del uniquecardidcity['union']

In [12]:
#data processing historical and new merchant data
logger.info('process historical and new merchant datasets')
for df in [historical_trans_df, new_merchant_trans_df]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['category_3'] = df['category_3'].map({'A':0, 'B':1, 'C':2}) 
    df['month_diff'] = ((pd.datetime(2012,4,1) - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    # Reference_date는 여기서 201903과 같은 형식으로 계산됩니다. 
    df['reference_date'] = (df['year']+(df['month'] - df['month_lag'])//12)*100 + (((df['month'] - df['month_lag'])%12) + 1)*1

    #3.691
    #df['installments'].replace(-1, np.nan,inplace=True)
    #df['installments'].replace(999, np.nan,inplace=True)
    
    #-1은 NAN을 상징하고 999는 12보다 큰 것을 의미하지 않을까???
    #df['installments'].replace(-1, np.nan,inplace=True)
    #df['installments'].replace(999, 13, inplace=True)

    # trim
    # 3.691단일 커널의 코드를 가져왔고, amount의 max도 중요하다고 생각해서 전처리는 하지 않았습니다. 
    #df['purchase_amount'] = df['purchase_amount'].apply(lambda x: min(x, 0.8))
    df['price'] = df['purchase_amount'] / df['installments']
    df['duration'] = df['purchase_amount']*df['month_diff']
    df['amount_month_ratio'] = df['purchase_amount']/df['month_diff']

[INFO]2019-02-06 10:26:01,430:main:process historical and new merchant datasets


점수 잘 안나옴. 
https://www.kaggle.com/prashanththangavel/c-ustomer-l-ifetime-v-alue
hist = historical_trans_df[['card_id','purchase_date','purchase_amount']]
hist = hist.sort_values(by=['card_id', 'purchase_date'], ascending=[True, True])

from datetime import datetime

z = hist.groupby('card_id')['purchase_date'].max().reset_index()
q = hist.groupby('card_id')['purchase_date'].min().reset_index()

z.columns = ['card_id', 'Max']
q.columns = ['card_id', 'Min']

## Extracting current timestamp
curr_date = pd.datetime(2012,4,1)

rec = pd.merge(z,q,how = 'left',on = 'card_id')
rec['Min'] = pd.to_datetime(rec['Min'])
rec['Max'] = pd.to_datetime(rec['Max'])

## Time value 
rec['Recency'] = (curr_date - rec['Max']).astype('timedelta64[D]') ## current date - most recent date

## Recency value
rec['Time'] = (rec['Max'] - rec['Min']).astype('timedelta64[D]') ## Age of customer, MAX - MIN

rec = rec[['card_id','Time','Recency']]

## Frequency
freq = hist.groupby('card_id').size().reset_index()
freq.columns = ['card_id', 'Frequency']
freq.head()

## Monitary
mon = hist.groupby('card_id')['purchase_amount'].sum().reset_index()
mon.columns = ['card_id', 'Monitary']
mon.head()

final = pd.merge(freq,mon,how = 'left', on = 'card_id')
final = pd.merge(final,rec,how = 'left', on = 'card_id')

final['historic_CLV'] = final['Frequency'] * final['Monitary'] 
final['AOV'] = final['Monitary']/final['Frequency'] ## AOV - Average order value (i.e) total_purchase_amt/total_trans
final['Predictive_CLV'] = final['Time']*final['AOV']*final['Monitary']*final['Recency'] 
historical_trans_df = pd.merge(historical_trans_df,final,on='card_id',how='left')

del historical_trans_df['Frequency']
del final
del mon
del freq
del rec
del z
del q
del curr_date
del hist
gc.collect()

In [13]:
# 이 부분은 카드가 사용되어지고 다음 카드가 사용되어지기 까지의 시간을 계산한 것 입니다. (diff)
# 메모리 문제 때문에 card_id만 가지고 했는데 다른 상점 id를 활용하면 더 좋을 것 같습니다. 
# 중간에 1440으로 나눠주는데 이는 분으로 계산한 것을 day로 바꿔주기 위함입니다. 
logger.info('process frequency of date cusotmer comeback by historical')

df = historical_trans_df[['card_id', 'purchase_date']]
df.sort_values(['card_id','purchase_date'], inplace=True) 
df['purchase'] = df.groupby(['card_id'])['purchase_date'].agg(['diff']).dropna(axis=0).astype('timedelta64[m]')
df['purchase'] = df['purchase'] //1440 #몇 일마다 사람이 방문하는지를 반영. 
del df['purchase_date']
#del df['subsector_id']

aggs = {}
aggs['purchase'] = ['min','max','mean','std','median']

df = df.groupby('card_id')['purchase'].agg(aggs).reset_index()

new_columns = ['card_id']
new_columns1 = create_new_columns('hist_freq',aggs)

for i in new_columns1:
    new_columns.append(i)

df.columns = new_columns

train_df = train_df.merge(df, on='card_id', how='left')
test_df = test_df.merge(df, on='card_id', how='left')
del df
gc.collect()

[INFO]2019-02-06 10:26:47,787:main:process frequency of date cusotmer comeback by historical


112

In [14]:
logger.info('process frequency of date cusotmer comeback by new')

df = new_merchant_trans_df[['card_id', 'purchase_date']]
df.sort_values(['card_id','purchase_date'], inplace=True) 
df['purchase'] = df.groupby(['card_id'])['purchase_date'].agg(['diff']).dropna(axis=0).astype('timedelta64[m]')
df['purchase'] = df['purchase'] //1440 #몇 일마다 사람이 방문하는지를 반영. 
del df['purchase_date']
#del df['subsector_id']

aggs = {}
aggs['purchase'] = ['min','max','mean','std','median']

df = df.groupby('card_id')['purchase'].agg(aggs).reset_index()

new_columns = ['card_id']
new_columns1 = create_new_columns('new_hist_freq',aggs)

for i in new_columns1:
    new_columns.append(i)

df.columns = new_columns

train_df = train_df.merge(df, on='card_id', how='left')
test_df = test_df.merge(df, on='card_id', how='left')
del df,new_columns1
gc.collect()

[INFO]2019-02-06 10:34:41,288:main:process frequency of date cusotmer comeback by new


112

In [15]:
# 기존에는 card_id를 기준으로 평균을 내었는데 이번에는 거꾸로 Reference를 기준으로 aggregation을 해봤습니다. 
logger.info('process reference_date by hist')

historical_trans_df_re = historical_trans_df[['reference_date','purchase_amount','authorized_flag','month_lag']]

aggs = {}
aggs['purchase_amount'] = ['min','max','mean','sum','std','median']
aggs['authorized_flag'] = ['min','max','mean','sum','std','median']

historical_trans_df_re = historical_trans_df_re.groupby(['reference_date'])[['purchase_amount','authorized_flag']].agg(aggs).reset_index()

new_columns = ['hist_reference_date_median']
new_columns1 = create_new_columns('hist_reference',aggs)

for i in new_columns1:
    new_columns.append(i)

historical_trans_df_re.columns = new_columns
del new_columns1
gc.collect();

[INFO]2019-02-06 10:36:13,279:main:process reference_date by hist


In [16]:
logger.info('process reference_date by new')

new_merchant_trans_df_re = new_merchant_trans_df[['reference_date','purchase_amount']]

aggs = {}
aggs['purchase_amount'] = ['max','mean','std','median']


new_merchant_trans_df_re = new_merchant_trans_df_re.groupby(['reference_date'])['purchase_amount'].agg(aggs).reset_index()

new_columns = ['hist_reference_date_median']
new_columns1 = create_new_columns('new_hist_reference',aggs)

for i in new_columns1:
    new_columns.append(i)

new_merchant_trans_df_re.columns = new_columns
del new_columns1
gc.collect();

[INFO]2019-02-06 10:36:22,042:main:process reference_date by new


In [17]:
# month_lag를 활용하여 purchase_amount에 가중치를 준 것입니다. 
# 이 부분을 더 개선시킬 수 있을 것 같은데 잘 안되는 중 입니다 ㅠㅠ...
historical_trans_df1 = historical_trans_df[['card_id','month_lag','purchase_amount']]
historical_trans_df3 = historical_trans_df1.groupby(['card_id','month_lag'])['purchase_amount'].agg({'count','mean'}).reset_index()
historical_trans_df3.columns = ['card_id','month_lag','month_lag_cnt','month_lag_amount_mean']
historical_trans_df3['month_lag_cnt'] = historical_trans_df3['month_lag_cnt']/(1-historical_trans_df3['month_lag']) 
historical_trans_df3['month_lag_amount_mean'] = historical_trans_df3['month_lag_amount_mean']/(1-historical_trans_df3['month_lag'])
del historical_trans_df3['month_lag']

aggs = {}
aggs['month_lag_cnt'] = ['min','max','mean','sum','std']
aggs['month_lag_amount_mean'] = ['min','max','mean','sum','std']


historical_trans_df3 = historical_trans_df3.groupby(['card_id']).agg(aggs).reset_index()

new_columns = ['card_id']
new_columns1 = create_new_columns('hist_weight',aggs)

for i in new_columns1:
    new_columns.append(i)
    
historical_trans_df3.columns = new_columns

del historical_trans_df1

#merge with train, test
train_df = train_df.merge(historical_trans_df3,on='card_id',how='left')
test_df = test_df.merge(historical_trans_df3,on='card_id',how='left')
del historical_trans_df3,new_columns1,new_columns
gc.collect();

In [18]:
#define aggregations with historical_trans_df
logger.info('Aggregate historical trans')
aggs = {}

for col in ['subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
for col in ['month', 'hour', 'weekofyear', 'dayofweek', 'year']:
    aggs[col] = ['nunique', 'mean', 'min', 'max']


aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean', 'min', 'max', 'var']
aggs['authorized_flag'] = ['sum', 'mean', 'min', 'max']
aggs['weekend'] = ['sum', 'mean', 'min', 'max']
aggs['category_1'] = ['sum', 'mean', 'min', 'max']
#aggs['category_2'] = ['sum', 'mean', 'min', 'max']
#aggs['category_3'] = ['sum', 'mean', 'min', 'max']
aggs['card_id'] = ['size', 'count']
aggs['reference_date'] = ['median']

## 아래 부분이 3.691커널에서 가져온 코드입니다. 

aggs['duration']=['mean','min','max','var','skew']
aggs['amount_month_ratio']=['mean','min','max','var','skew']
aggs['price'] = ['sum','mean','max','min','var']

## Version3 Encoding
aggs['city_id_Frequency'] = ['mean','sum','var','median']
aggs['merchant_category_id_Frequency'] = ['mean','sum','var','median']
aggs['merchant_id_Frequency'] = ['mean','sum','var','median']
aggs['state_id_Frequency'] = ['mean','sum','var','median']
aggs['subsector_id_Frequency'] = ['mean','sum','var','median']

[INFO]2019-02-06 10:36:30,904:main:Aggregate historical trans


In [19]:
new_columns = create_new_columns('hist',aggs)
historical_trans_group_df = historical_trans_df.groupby('card_id').agg(aggs)
historical_trans_group_df.columns = new_columns
historical_trans_group_df.reset_index(drop=False,inplace=True)
historical_trans_group_df['hist_purchase_date_diff'] = (historical_trans_group_df['hist_purchase_date_max'] - historical_trans_group_df['hist_purchase_date_min']).dt.days
historical_trans_group_df['hist_purchase_date_average'] = historical_trans_group_df['hist_purchase_date_diff']/historical_trans_group_df['hist_card_id_size']
historical_trans_group_df['hist_purchase_date_uptonow'] = (pd.datetime(2012,4,1) - historical_trans_group_df['hist_purchase_date_max']).dt.days
historical_trans_group_df['hist_purchase_date_uptomin'] = (pd.datetime(2012,4,1) - historical_trans_group_df['hist_purchase_date_min']).dt.days
#merge with train, test
train_df = train_df.merge(historical_trans_group_df,on='card_id',how='left')
test_df = test_df.merge(historical_trans_group_df,on='card_id',how='left')

#cleanup memory
del historical_trans_group_df; gc.collect()

35

In [20]:
#define aggregations with new_merchant_trans_df 
logger.info('Aggregate new merchant trans')
aggs = {}
for col in ['subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']
for col in ['month', 'hour', 'weekofyear', 'dayofweek', 'year']:
    aggs[col] = ['nunique', 'mean', 'min', 'max']

    
aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean', 'max', 'min', 'var']
aggs['weekend'] = ['sum', 'mean', 'min', 'max']
aggs['category_1'] = ['sum', 'mean', 'min', 'max']
aggs['authorized_flag'] = ['sum']
#aggs['category_2'] = ['sum', 'mean', 'min', 'max']
#aggs['category_3'] = ['sum', 'mean', 'min', 'max']
aggs['card_id'] = ['size']
aggs['reference_date'] = ['median']
##3.691
aggs['duration']=['mean','min','max','var','skew']
aggs['amount_month_ratio']=['mean','min','max','var','skew']
aggs['price'] = ['sum','mean','max','min','var']

## Version3 Encoding
aggs['city_id_Frequency'] = ['mean','sum','var','median']
aggs['merchant_category_id_Frequency'] = ['mean','sum','var','median']
aggs['merchant_id_Frequency'] = ['mean','sum','var','median']
aggs['state_id_Frequency'] = ['mean','sum','var','median']
aggs['subsector_id_Frequency'] = ['mean','sum','var','median']

[INFO]2019-02-06 10:41:11,145:main:Aggregate new merchant trans


In [21]:
new_columns = create_new_columns('new_hist',aggs)
new_merchant_trans_group_df = new_merchant_trans_df.groupby('card_id').agg(aggs)
new_merchant_trans_group_df.columns = new_columns
new_merchant_trans_group_df.reset_index(drop=False,inplace=True)
new_merchant_trans_group_df['new_hist_purchase_date_diff'] = (new_merchant_trans_group_df['new_hist_purchase_date_max'] - new_merchant_trans_group_df['new_hist_purchase_date_min']).dt.days
new_merchant_trans_group_df['new_hist_purchase_date_average'] = new_merchant_trans_group_df['new_hist_purchase_date_diff']/new_merchant_trans_group_df['new_hist_card_id_size']
new_merchant_trans_group_df['new_hist_purchase_date_uptonow'] = (pd.datetime(2012,4,1) - new_merchant_trans_group_df['new_hist_purchase_date_max']).dt.days
new_merchant_trans_group_df['new_hist_purchase_date_uptomin'] = (pd.datetime(2012,4,1) - new_merchant_trans_group_df['new_hist_purchase_date_min']).dt.days
#merge with train, test
train_df = train_df.merge(new_merchant_trans_group_df,on='card_id',how='left')
test_df = test_df.merge(new_merchant_trans_group_df,on='card_id',how='left')

#clean-up memory
del new_merchant_trans_group_df; gc.collect()
del historical_trans_df; gc.collect()
del new_merchant_trans_df; gc.collect()

14

In [22]:
#merge with train, test
train_df = train_df.merge(historical_trans_df_re,on='hist_reference_date_median',how='left')
test_df = test_df.merge(historical_trans_df_re,on='hist_reference_date_median',how='left')

train_df = train_df.merge(new_merchant_trans_df_re,on='hist_reference_date_median',how='left')
test_df = test_df.merge(new_merchant_trans_df_re,on='hist_reference_date_median',how='left')
del historical_trans_df_re
del new_merchant_trans_df_re
gc.collect()

42

In [23]:
#process train
logger.info('Process train')
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1
train_df['outliers'].value_counts()

logger.info('Process train and test')
## process both train and test
for df in [train_df, test_df]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['dayofyear'] = df['first_active_month'].dt.dayofyear
    df['quarter'] = df['first_active_month'].dt.quarter
    df['is_month_start'] = df['first_active_month'].dt.is_month_start
    df['month'] = df['first_active_month'].dt.month
    df['year'] = df['first_active_month'].dt.year
    df['first_active_month1'] = 100*df['year']+df['month']
    
    df['elapsed_time'] = (pd.datetime(2012,4,1) - df['first_active_month']).dt.days
    
    #hist_reference_date_median은 201901과 같은 형식인데 이를 2019-01로 바꿔서 pd.to_datetime이 동작핟록 형식을 바꿔주었습니다. 
    df['hist_reference_date_median'] = df['hist_reference_date_median'].astype(str)
    df['hist_reference_date_median'] = df['hist_reference_date_median'].apply(lambda x: x[0:4]+'-'+x[4:6])
    df['hist_reference_date_median'] = pd.to_datetime(df['hist_reference_date_median'])
    
    
    df['ref_year'] =df['hist_reference_date_median'].dt.year
    df['ref_month'] =df['hist_reference_date_median'].dt.month
    df['reference_month1'] = 100*df['ref_year']+df['ref_month']
    
#    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
#    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
#    df['days_feature3'] = df['elapsed_time'] * df['feature_3']

#    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
#    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
#    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']    

    ## 3.691에서 가져온 코드입니다. 
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']
    df['purchase_amount_mean'] = df['new_hist_purchase_amount_mean']+df['hist_purchase_amount_mean']
    df['purchase_amount_max'] = df['new_hist_purchase_amount_max']+df['hist_purchase_amount_max']
    df['purchase_amount_min'] = df['new_hist_purchase_amount_min']+df['hist_purchase_amount_min']
    df['purchase_amount_sum_ratio'] = df['new_hist_purchase_amount_sum']/df['hist_purchase_amount_sum']
    
    #VERSION24에서 RATIO추가
    #VERSION25, 26차이. 
    df['nh_purchase_amount_mean_ratio'] = df['new_hist_purchase_amount_mean']/df['hist_purchase_amount_mean']

    ## 이 부분은 거래의 기간을 계산한 값들입니다. ratio로도 활용하면 의미가 있을 것 같지만 시도는 안해봤습니다. 
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days 
    df['hist_last_buy'] = (df['hist_purchase_date_max'] - df['first_active_month']).dt.days 
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days 
    df['new_hist_last_buy'] = (df['new_hist_purchase_date_max'] - df['first_active_month']).dt.days 
    
    ## 마찬가지로 거래의 기간을 계산한 값들입니다. 위에는 first_active_month가 기준이고 아래는 reference_date가 기준입니다. 
    df['year_month'] = df['year']*100 + df['month']
    df['hist_diff_reference_date_first'] = (df['hist_reference_date_median'] - df['first_active_month']).dt.days 
    df['hist_diff_reference_date_min'] = (df['hist_reference_date_median'] - df['hist_purchase_date_min']).dt.days 
    df['hist_diff_reference_date_max'] = (df['hist_reference_date_median'] - df['hist_purchase_date_max']).dt.days 
    df['new_hist_diff_reference_date_min'] = (df['hist_reference_date_median'] - df['new_hist_purchase_date_min']).dt.days 
    df['new_hist_diff_reference_date_max'] = (df['hist_reference_date_median'] - df['new_hist_purchase_date_max']).dt.days 
    
    ## 거래의 기간을 계산한 값입니다. 
    df['hist_diff_first_last'] = df['hist_last_buy'] - df['hist_first_buy'] 
    df['new_hist_diff_first_last'] = df['new_hist_last_buy'] - df['new_hist_first_buy'] 
    
    #version11
    ## 거래기간동안 얼마나 거래가 이루어진지 평균을 내본 값입니다. 
    df['hist_diff_first_last_purchase'] = df['hist_purchase_amount_sum'] / df['hist_diff_first_last'] 
    df['new_hist_diff_first_last_purchase'] = df['new_hist_purchase_amount_sum'] / df['new_hist_diff_first_last'] 
    
    #VERSION24에서 RATIO추가
    #VERSION30에서 추가. 
    df['nh_purchase_mean_average_ratio'] = df['new_hist_diff_first_last_purchase']/df['hist_diff_first_last_purchase'] #중요도 낮음.
    
    #VERSION25, 27차이. 
    df['nh_merchant_id_nunique_ratio'] = df['new_hist_merchant_id_nunique']/df['hist_merchant_id_nunique']
    
    #VERSION4 ID 갯수 비율 추가
    #df['nh_city_id_nunique_ratio'] = df['new_hist_city_id_nunique']/df['hist_city_id_nunique']
    #df['nh_state_id_nunique_ratio'] = df['new_hist_state_id_nunique']/df['hist_state_id_nunique']
    #CV점수 안좋아져서 제거했음. LB는 모름. 
    #del df['new_hist_city_id_nunique'], df['hist_city_id_nunique']
    #del df['new_hist_state_id_nunique'], df['hist_state_id_nunique']
    #del df['nh_city_id_nunique_ratio'], df['nh_state_id_nunique_ratio']
    
    ## 위랑 동일
    df['hist_card_id_size_average'] = df['new_hist_card_id_size'] / df['hist_diff_first_last']
    df['new_hist_card_id_size_average'] = df['new_hist_card_id_size'] / df['new_hist_diff_first_last']
    
    # VERSION24에서 RATIO추가
    # VERSION31에서 테스트중..
    df['nh_card_id_size_average_ratio'] = df['new_hist_card_id_size_average']/df['hist_card_id_size_average'] #중요도 낮음. 
    
    #VERSION25, 28차이. 
    df['nh_freq_purchase_mean_ratio'] = df['new_hist_freq_purchase_mean']/df['hist_freq_purchase_mean']
    
    # VERSION32에서 테스트중.. 
    df['nh_category_1_sum_ratio'] = df['new_hist_category_1_sum']/df['hist_category_1_sum'] #중요도 낮음. 
    #df['nh_category_1_mean_ratio'] = df['new_hist_category_1_mean']/df['hist_category_1_mean'] #중요도 낮음. 

    ## 마찬가지로 거래의 기간을 계산 hist와 new와의 관계
    df['diff_new_hist_date_min_max'] = (df['new_hist_purchase_date_min'] - df['hist_purchase_date_max']).dt.days 
    df['diff_new_hist_date_max_max'] = (df['new_hist_purchase_date_max'] - df['hist_purchase_date_max']).dt.days    
    df['diff_new_hist_date_max_min'] = (df['new_hist_purchase_date_max'] - df['hist_purchase_date_min']).dt.days 

    #Version14 : 중요한한 변수를 나눠서 상호작용하도록 만듬.
    df['diff_new_hist_date_max_amount_max'] = df['new_hist_purchase_amount_max']/df['diff_new_hist_date_max_max']
    
    
    df['hist_flag_ratio'] = df['hist_authorized_flag_sum'] / df['hist_card_id_size']

    ### LB 3.691 커널에서 추가한 부분. 
    df['installments_total'] = df['new_hist_installments_sum']+df['hist_installments_sum']
    df['installments_mean'] = df['new_hist_installments_mean']+df['hist_installments_mean']
    df['installments_max'] = df['new_hist_installments_max']+df['hist_installments_max']
    df['installments_ratio'] = df['new_hist_installments_sum']/df['hist_installments_sum']
    
    df['price_total'] = df['purchase_amount_total'] / df['installments_total']
    df['price_mean'] = df['purchase_amount_mean'] / df['installments_mean']
    df['price_max'] = df['purchase_amount_max'] / df['installments_max']
    df['duration_mean'] = df['new_hist_duration_mean']+df['hist_duration_mean']
    # VERSION24에서 RATIO추가
    #VERSION25, 29차이. 
    #df['duration_ratio'] = df['new_hist_duration_mean']/df['hist_duration_mean']
    
    df['duration_min'] = df['new_hist_duration_min']+df['hist_duration_min']
    df['duration_max'] = df['new_hist_duration_max']+df['hist_duration_max']
    df['amount_month_ratio_mean']=df['new_hist_amount_month_ratio_mean']+df['hist_amount_month_ratio_mean']
    df['amount_month_ratio_min']=df['new_hist_amount_month_ratio_min']+df['hist_amount_month_ratio_min']
    df['amount_month_ratio_max']=df['new_hist_amount_month_ratio_max']+df['hist_amount_month_ratio_max']
    df['new_CLV'] = df['new_hist_card_id_size'] * df['new_hist_purchase_amount_sum'] / df['new_hist_month_diff_mean']
    df['hist_CLV'] = df['hist_card_id_size'] * df['hist_purchase_amount_sum'] / df['hist_month_diff_mean']
    df['CLV_ratio'] = df['new_CLV'] / df['hist_CLV']
    
   
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
        
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    
    del df['year']
    del df['year_month']
    del df['new_hist_reference_date_median']

    
for f in ['feature_1','feature_2','feature_3']: 
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)
    
#for df in [train_df, test_df]:    
#    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
#    df['feature_mean'] = df['feature_sum']/3
#    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
#    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
#    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

## 유니크 값이 1이면 제거하는 코드입니다. 
for col in train_df.columns:
    if train_df[col].nunique() == 1:
        print(col)
        del train_df[col]
        del test_df[col]
##

train_columns = [c for c in train_df.columns if c not in ['card_id', 'first_active_month','target','outliers','hist_reference_date_median']]

target = train_df['target']
#del train_df['target']

[INFO]2019-02-06 10:43:12,475:main:Process train
[INFO]2019-02-06 10:43:12,585:main:Process train and test


hist_year_min
hist_authorized_flag_max
hist_reference_authorized_flag_min
hist_reference_authorized_flag_max
hist_reference_authorized_flag_median
is_month_start


from scipy.stats import ks_2samp
from tqdm import tqdm
list_p_value =[]

for i in tqdm(train_columns):
    list_p_value.append(ks_2samp(test_df[i] , train_df[i])[1])

Se = pd.Series(list_p_value, index = train_columns).sort_values() 
list_discarded = list(Se[Se < .1].index)

for i in list_discarded:
    train_columns.remove(i)

In [33]:
train = train_df.copy()
train = train.loc[train['target']>-30]
target = train['target']
del train['target']

In [36]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.015,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 24,
         "seed": 6}

#prepare fit model with cross-validation
np.random.seed(2019)

folds = KFold(n_splits=9, shuffle=True, random_state=4950)
oof = np.zeros(len(train))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train)):
    strLog = "fold {}".format(fold_+1)
    print(strLog)
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    #feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_columns
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #predictions
    predictions += clf.predict(test_df[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    
cv_score = np.sqrt(mean_squared_error(oof, target))
print(cv_score)
withoutoutlier_predictions = predictions.copy()

fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.58527	valid_1's rmse: 1.57841
[200]	training's rmse: 1.55738	valid_1's rmse: 1.55564
[300]	training's rmse: 1.5434	valid_1's rmse: 1.54756
[400]	training's rmse: 1.53329	valid_1's rmse: 1.54375
[500]	training's rmse: 1.52479	valid_1's rmse: 1.54166
[600]	training's rmse: 1.51721	valid_1's rmse: 1.54062
[700]	training's rmse: 1.51032	valid_1's rmse: 1.53997
[800]	training's rmse: 1.50383	valid_1's rmse: 1.53939
[900]	training's rmse: 1.49772	valid_1's rmse: 1.53926
[1000]	training's rmse: 1.49179	valid_1's rmse: 1.53877
[1100]	training's rmse: 1.48597	valid_1's rmse: 1.53846
[1200]	training's rmse: 1.48039	valid_1's rmse: 1.53824
[1300]	training's rmse: 1.47479	valid_1's rmse: 1.53817
[1400]	training's rmse: 1.46938	valid_1's rmse: 1.53816
[1500]	training's rmse: 1.46396	valid_1's rmse: 1.53816
Early stopping, best iteration is:
[1414]	training's rmse: 1.4686	valid_1's rmse: 1.53807
fold 2
Tra

[500]	training's rmse: 1.5217	valid_1's rmse: 1.5663
[600]	training's rmse: 1.51427	valid_1's rmse: 1.56529
[700]	training's rmse: 1.5074	valid_1's rmse: 1.56461
[800]	training's rmse: 1.50092	valid_1's rmse: 1.5642
[900]	training's rmse: 1.49481	valid_1's rmse: 1.56414
Early stopping, best iteration is:
[863]	training's rmse: 1.4971	valid_1's rmse: 1.56399
1.553231146572967


In [37]:
model_without_outliers = pd.DataFrame({"card_id":test_df["card_id"].values})
model_without_outliers["target"] = withoutoutlier_predictions

In [39]:
model_without_outliers.to_csv('hyeonwoo_without_outlier.csv',index=False)

### Outlier Model

In [24]:
train = train_df.copy()
target = train['outliers']
del train['target']
del train['outliers']

In [25]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.6,
         "bagging_freq": 1,
         "bagging_fraction": 0.7 ,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 24,
         "random_state": 6}

folds = KFold(n_splits=9, shuffle=True, random_state=4950)
oof = np.zeros(len(train))
predictions = np.zeros(len(test_df))

#start = time.time()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold {}".format(fold_+1))

    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    predictions += clf.predict(test_df[train_columns], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(log_loss(target, oof)))

fold 1
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0442295	valid_1's binary_logloss: 0.0473734
[200]	training's binary_logloss: 0.040707	valid_1's binary_logloss: 0.0452727
[300]	training's binary_logloss: 0.0387534	valid_1's binary_logloss: 0.0446588
[400]	training's binary_logloss: 0.0374018	valid_1's binary_logloss: 0.0445221
[500]	training's binary_logloss: 0.0359787	valid_1's binary_logloss: 0.0445202
[600]	training's binary_logloss: 0.0346823	valid_1's binary_logloss: 0.0445798
Early stopping, best iteration is:
[441]	training's binary_logloss: 0.0368117	valid_1's binary_logloss: 0.0444846
fold 2
Training until validation scores don't improve for 200 rounds.
[100]	training's binary_logloss: 0.0442358	valid_1's binary_logloss: 0.0465009
[200]	training's binary_logloss: 0.0407708	valid_1's binary_logloss: 0.0447546
[300]	training's binary_logloss: 0.0388141	valid_1's binary_logloss: 0.0443011
[400]	training's binary_logloss: 0.0

NameError: name 'log_loss' is not defined

In [27]:
print("CV score: {:<8.5f}".format(log_loss(target, oof)))

CV score: 0.04354 


In [ ]:
df_outlier_prob = pd.DataFrame({"card_id":test_df["card_id"].values})
df_outlier_prob["target"] = predictions
df_outlier_prob.sort_values('target',ascending=False)